# 1. Import Required Libraries

In [34]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# Set random seed for reproducibility
np.random.seed(42)


# Import additional libraries for hyperparameter tuning
from sklearn.metrics import precision_score, recall_score, classification_report
import optuna
from optuna.samplers import TPESampler

# Suppress Optuna logging
optuna.logging.set_verbosity(optuna.logging.WARNING)

# 2. Load Data set

Helper functions created successfully!
Ready to train SVM models with hyperparameter tuning using Optuna.


### 4.I.a. Building Pipeline with StandardScaler and SVM model

Hyperparameters are tuned using Optuna. 


In [8]:
# Baseline SVM with default parameters
baseline_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(
        kernel='rbf',
        random_state=42
    ))
])

# Train and evaluate
baseline_result = evaluate_model(
    baseline_svm, 
    "Baseline SVM (default params)", 
    X_train, X_test, y_train, y_test
)

c:\Users\jhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


c:\Users\jhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



=== Baseline SVM (default params) ===
Training Accuracy: 0.6084 (60.84%)
Testing Accuracy: 0.5705 (57.05%)
Training F1-macro: 0.2435
Testing F1-macro: 0.2266
Precision (macro): 0.2501
Recall (macro): 0.2289
--------------------------------------------------


c:\Users\jhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


### 4.I.b. Hyperparameter Tuning with Optuna - RBF Kernel

Use Optuna to automatically find the best hyperparameters for an RBF kernel SVM.

In [9]:
def objective_rbf(trial):
    """Optuna objective function for RBF kernel SVM"""
    # Suggest hyperparameters
    C = trial.suggest_float('C', 0.1, 100.0, log=True)
    gamma = trial.suggest_float('gamma', 0.001, 1.0, log=True)
    
    # Create pipeline
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('svm', SVC(kernel='rbf', C=C, gamma=gamma, random_state=42))
    ])
    
    # Train and evaluate
    pipeline.fit(X_train, y_train)
    y_val_pred = pipeline.predict(X_test)
    
    # Return F1-macro score (maximize)
    return f1_score(y_test, y_val_pred, average='macro')

# Create study and optimize
print("Optimizing RBF SVM with Optuna...")
study_rbf = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=42)
)
study_rbf.optimize(objective_rbf, n_trials=50, show_progress_bar=True)

# Get best parameters
best_params_rbf = study_rbf.best_params
print(f"\nBest parameters for RBF SVM:")
print(f"  C: {best_params_rbf['C']:.4f}")
print(f"  gamma: {best_params_rbf['gamma']:.4f}")
print(f"  Best F1-macro score: {study_rbf.best_value:.4f}")

# Train model with best parameters
optimized_rbf_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(
        kernel='rbf',
        C=best_params_rbf['C'],
        gamma=best_params_rbf['gamma'],
        random_state=42
    ))
])

# Evaluate
optimized_rbf_result = evaluate_model(
    optimized_rbf_svm,
    "Optuna-Optimized RBF SVM",
    X_train, X_test, y_train, y_test
)

Optimizing RBF SVM with Optuna...


Optimizing RBF SVM with Optuna...


  0%|          | 0/50 [00:00<?, ?it/s]

Optimizing RBF SVM with Optuna...


  0%|          | 0/50 [00:00<?, ?it/s]

...

c:\Users\jhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1406: DataConve


Best parameters for RBF SVM:
  C: 25.5292
  gamma: 0.5676
  Best F1-macro score: 0.4107


c:\Users\jhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



=== Optuna-Optimized RBF SVM ===
Training Accuracy: 0.9936 (99.36%)
Testing Accuracy: 0.6628 (66.28%)
Training F1-macro: 0.9963
Testing F1-macro: 0.4107
Precision (macro): 0.4383
Recall (macro): 0.3931
--------------------------------------------------


c:\Users\jhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


### 4.I.c. Hyperparameter Tuning with Optuna - Linear Kernel

Optimize a linear kernel SVM using Optuna.

In [10]:
def objective_linear(trial):
    """Optuna objective function for Linear kernel SVM"""
    # Suggest hyperparameters
    C = trial.suggest_float('C', 0.01, 100.0, log=True)
    
    # Create pipeline
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('svm', SVC(kernel='linear', C=C, random_state=42))
    ])
    
    # Train and evaluate
    pipeline.fit(X_train, y_train)
    y_val_pred = pipeline.predict(X_test)
    
    # Return F1-macro score
    return f1_score(y_test, y_val_pred, average='macro')

# Optimize
print("Optimizing Linear SVM with Optuna...")
study_linear = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=42)
)
study_linear.optimize(objective_linear, n_trials=30, show_progress_bar=True)

# Get best parameters
best_params_linear = study_linear.best_params
print(f"\nBest parameters for Linear SVM:")
print(f"  C: {best_params_linear['C']:.4f}")
print(f"  Best F1-macro score: {study_linear.best_value:.4f}")

# Train with best parameters
optimized_linear_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(
        kernel='linear',
        C=best_params_linear['C'],
        random_state=42
    ))
])

# Evaluate
optimized_linear_result = evaluate_model(
    optimized_linear_svm,
    "Optuna-Optimized Linear SVM",
    X_train, X_test, y_train, y_test
)

Optimizing Linear SVM with Optuna...


Optimizing Linear SVM with Optuna...


  0%|          | 0/30 [00:00<?, ?it/s]

Optimizing Linear SVM with Optuna...


  0%|          | 0/30 [00:00<?, ?it/s]

...

c:\Users\jhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1406: DataConve


Best parameters for Linear SVM:
  C: 0.0421
  Best F1-macro score: 0.1669


c:\Users\jhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



=== Optuna-Optimized Linear SVM ===
Training Accuracy: 0.5365 (53.65%)
Testing Accuracy: 0.5188 (51.88%)
Training F1-macro: 0.1733
Testing F1-macro: 0.1669
Precision (macro): 0.1528
Recall (macro): 0.1896
--------------------------------------------------


c:\Users\jhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


### 4.I.d. Hyperparameter Tuning with Optuna - Polynomial Kernel

Optimize a polynomial kernel SVM using Optuna.

In [ ]:
def objective_poly(trial):
    """Optuna objective function for Polynomial kernel SVM"""
    # Suggest hyperparameters
    C = trial.suggest_float('C', 0.1, 100.0, log=True)
    degree = trial.suggest_int('degree', 2, 5)
    gamma = trial.suggest_float('gamma', 0.001, 1.0, log=True)
    coef0 = trial.suggest_float('coef0', 0.0, 10.0)
    
    # Create pipeline
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('svm', SVC(
            kernel='poly',
            C=C,
            degree=degree,
            gamma=gamma,
            coef0=coef0,
            random_state=42
        ))
    ])
    
    # Train and evaluate
    pipeline.fit(X_train, y_train)
    y_val_pred = pipeline.predict(X_test)
    
    # Return F1-macro score
    return f1_score(y_test, y_val_pred, average='macro')

# Optimize
print("Optimizing Polynomial SVM with Optuna...")
study_poly = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=42)
)
study_poly.optimize(objective_poly, n_trials=15, show_progress_bar=True)

# Get best parameters
best_params_poly = study_poly.best_params
print(f"\nBest parameters for Polynomial SVM:")
print(f"  C: {best_params_poly['C']:.4f}")
print(f"  degree: {best_params_poly['degree']}")
print(f"  gamma: {best_params_poly['gamma']:.4f}")
print(f"  coef0: {best_params_poly['coef0']:.4f}")
print(f"  Best F1-macro score: {study_poly.best_value:.4f}")

# Train with best parameters
optimized_poly_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(
        kernel='poly',
        C=best_params_poly['C'],
        degree=best_params_poly['degree'],
        gamma=best_params_poly['gamma'],
        coef0=best_params_poly['coef0'],
        random_state=42
    ))
])

# Evaluate
optimized_poly_result = evaluate_model(
    optimized_poly_svm,
    "Optuna-Optimized Polynomial SVM",
    X_train, X_test, y_train, y_test
)

Optimizing Polynomial SVM with Optuna...


  0%|          | 0/50 [00:00<?, ?it/s]

c:\Users\jhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1406: DataConve

# 6. Further Answers:


<mark style="background-color: #fff000;">Item b. Were you able to improve against the result in the paper? </mark>

The overall accuracies reported in the paper are 62.4% (red) and 64.6% (white). 

The best model produced an accurary of 


<mark style="background-color: #fff000;">Item b. Based on your results, discuss the difference of treating this problem as classification or regression. How will this decision impact the users of your model?</mark>

Original Paper: 
https://www.sciencedirect.com/science/article/pii/S0167923609001377



In [ ]:
# Display an image
idisplay(Image(filename=image_path))
mage_path = "patAI2a\wi/e_manu_table2.1png# Replace with your image path
didisplay(Image(filename=image_path))
splay(Image(filename=image_path))# Get best model
best_model_name = best_test_f1[0]
best_model_results = best_test_f1[1]
best_pipeline = best_model_results['pipeline']

# Make predictions
y_train_pred = best_pipeline.predict(X_train)
y_test_pred = best_pipeline.predict(X_test)

print("=" * 100)
print(f"DETAILED PERFORMANCE REPORT: {best_model_name}")
print("=" * 100)

# Training data metrics
print("\n📊 TRAINING DATA METRICS:")
print(f"  Accuracy:  {best_model_results['train_accuracy']:.4f} ({best_model_results['train_accuracy']*100:.2f}%)")
print(f"  F1-Score:  {best_model_results['train_f1_macro']:.4f}")

# Testing data metrics
print("\n📊 TESTING DATA METRICS:")
print(f"  Accuracy:   {best_model_results['test_accuracy']:.4f} ({best_model_results['test_accuracy']*100:.2f}%)")
print(f"  Precision:  {best_model_results['test_precision']:.4f}")
print(f"  Recall:     {best_model_results['test_recall']:.4f}")
print(f"  F1-Score:   {best_model_results['test_f1_macro']:.4f}")

# Performance gap
print("\n📈 GENERALIZATION:")
print(f"  Accuracy Gap: {best_model_results['train_accuracy'] - best_model_results['test_accuracy']:.4f}")
print(f"  F1-Score Gap: {best_model_results['train_f1_macro'] - best_model_results['test_f1_macro']:.4f}")

# Detailed classification report
print("\n📋 DETAILED CLASSIFICATION REPORT (Test Data):")
print(classification_report(y_test, y_test_pred))

# 7. Confusion Matrices

Visualize confusion matrices for training and testing data.

In [ ]:
# Plot confusion matrices
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Training data confusion matrix
cm_train = confusion_matrix(y_train, y_train_pred)
disp_train = ConfusionMatrixDisplay(
    confusion_matrix=cm_train,
    display_labels=best_pipeline['svm'].classes_
)
disp_train.plot(ax=ax1, cmap='Blues', values_format='d')
ax1.set_title(f'Training Data Confusion Matrix\n{best_model_name}', fontsize=14, pad=20)
ax1.grid(False)

# Testing data confusion matrix
cm_test = confusion_matrix(y_test, y_test_pred)
disp_test = ConfusionMatrixDisplay(
    confusion_matrix=cm_test,
    display_labels=best_pipeline['svm'].classes_
)
disp_test.plot(ax=ax2, cmap='Oranges', values_format='d')
ax2.set_title(f'Testing Data Confusion Matrix\n{best_model_name}', fontsize=14, pad=20)
ax2.grid(False)

plt.tight_layout()
plt.show()

print("\n📊 Confusion Matrix Analysis:")
print("• Diagonal elements represent correct predictions")
print("• Off-diagonal elements represent misclassifications")
print(f"• Training samples: {len(y_train)}")
print(f"• Testing samples: {len(y_test)}")

## 8. Optuna Optimization History

Visualize the optimization history to understand how Optuna found the best parameters.